In [ ]:
from mlflow.tracking import MlflowClient
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer
import mlflow.transformers

# Input arguments
model_dir = "/mnt/data/test-fe8a4e/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6"
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
revision = "fe8a4ea1ffedaf415f4da2f062534de366a451e6"
model_type = "LLM"

# Load the model and tokenizer from model_dir based on model_type
if model_type == "LLM":
    model = AutoModelForCausalLM.from_pretrained(model_dir)
else:
    model = AutoModel.from_pretrained(model_dir)
    
tokenizer = AutoTokenizer.from_pretrained(model_dir)
print("Loaded model and tokenizer")

with mlflow.start_run() as run:
    # Log the model
    mlflow.transformers.log_model(
        transformers_model={"model": model, "tokenizer": tokenizer},
        artifact_path="model",
        task="text-generation",
        input_example="Hello, how are you?",
        model_card=None,  # ✅ Skip HF model card fetch
        metadata={"hf_source": "local"},  # optional, to clarify source
    )
    print("Logged model")

    # TODO Check for metadata conflicts if the registered model already exists.
    # For example, the model_id must match the existing tag.

    # Register the model
    result = mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/model",
        name=model_id,
        description=f"{model_id} registered from Hugging Face",
    )
    print(f"Registered model={model_id}, version={result.version}")
    
client = MlflowClient()

# Tag the registered model
if result.version == 1:
    client.set_registered_model_tag(name=model_id, key="mlflow.domino.model_id", value=model_id)
    client.set_registered_model_tag(name=model_id, key="mlflow.domino.model_type", value=model_type)

# Tag the registered model version
client.set_model_version_tag(
    name=model_id,
    version=result.version,
    key="mlflow.domino.model_version",
    value=revision,
)
    

Device set to use cpu


Loaded model and tokenizer


2025/07/26 04:35:47 WARNING mlflow.transformers: The model card could not be retrieved from the hub due to Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/mnt/data/test-fe8a4e/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6'. Use `repo_type` argument if needed.
2025/07/26 04:35:47 WARNING mlflow.transformers: Unable to find license information for this model. Please verify permissible usage for the model you are storing prior to use.
2025/07/26 04:35:47 INFO mlflow.transformers.signature: Running model prediction to infer the model output signature with a timeout of 180 seconds. You can specify a different timeout by setting the environment variable MLFLOW_INPUT_EXAMPLE_INFERENCE_TIMEOUT.
/opt/conda/lib/python3.10/site-packages/mlflow/transformers/signature.py:150: FutureWarning: ``mlflow.transformers.signature.generate_signature_output`` is deprecated since 2.19.0. This method will be removed in a future release. Use `

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

Device set to use cpu


In [3]:
!ls -l /mnt/data/test-fe8a4e

total 4
drwxr-xr-x. 6 root root 6144 Jul 26 02:00 models--TinyLlama--TinyLlama-1.1B-Chat-v1.0
